In [ ]:
!pip install geopandas pandas shapely fiona pyproj openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

input_folder = '/content/drive/My Drive/Shapefiles'  # Folder containing input layers # @param {type:"string"}
zone_folder = '/content/drive/My Drive/Clip_To'  # Folder containing overlay zone layers # @param {type:"string"}
output_folder = os.path.join(input_folder, "Clipped_Output") #Folder to save # @param {type:"string"}
os.makedirs(output_folder, exist_ok=True)
print(f"Outputs will be saved in: {output_folder}")

print(f"Building layers folder: {input_folder}")
print(f"Zone layers folder: {zone_folder}")
print(f"Output folder: {output_folder}")


Processing vector files in: /content/drive/My Drive/Shapefiles
Enter the field name for Compactness Index (CI): area_m2
Enter the field name for building class: fclass
Results will be saved in: /content/drive/My Drive/Shapefiles/Average_CI_Outputs


In [ ]:
import glob
import geopandas as gpd
from concurrent.futures import ThreadPoolExecutor

input_extensions = ['*.shp', '*.gpkg', '*.geojson', '*.json', '*.kml', '*.gml']
zone_extensions = ['*.shp', '*.gpkg', '*.geojson', '*.json', '*.kml', '*.gml']

input_files = []
for ext in input_extensions:
    input_files.extend(glob.glob(os.path.join(input_folder, ext)))

zone_files = []
for ext in zone_extensions:
    zone_files.extend(glob.glob(os.path.join(zone_folder, ext)))

if not input_files:
    raise SystemExit("No building files found.")
if not zone_files:
    raise SystemExit("No zone files found.")

def clip_file(building_file, zone_file, output_folder, idx):
    try:
        b_gdf = gpd.read_file(building_file)
    except:
        print(f"Failed to read {building_file}")
        return

    try:
        z_gdf = gpd.read_file(zone_file)
    except:
        print(f"Failed to read {zone_file}")
        return

    try:
        clipped = gpd.overlay(b_gdf, z_gdf, how="intersection")
    except Exception as e:
        print(f"Clip failed for {building_file} with {zone_file}: {e}")
        return

    output_name = f"{os.path.splitext(os.path.basename(building_file))[0]}_clipped_{idx}.gpkg"
    output_path = os.path.join(output_folder, output_name)
    clipped.to_file(output_path, driver="GPKG")
    print(f"Saved clipped layer: {output_path}")

max_threads = min(8, len(input_files) * len(zone_files))

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    idx = 1
    for b_file in input_files:
        for z_file in zone_files:
            executor.submit(clip_file, b_file, z_file, output_folder, idx)
            idx += 1

print("Parallel batch clipping complete!")



File: NL_airports.gpkg

File: NL_dams.gpkg

File: NL_agriculture.gpkg

File: NL_parks.gpkg

File: NL_parking.gpkg
Skipping /content/drive/My Drive/Shapefiles/NL_coastal_marine_infrastructure.gpkg (fields not found)
          average_CI
fclass              
dam     19474.019467
                          average_CI
fclass                              
airport_international   10271.733712
airport_regional       404722.100445
apron                  172707.454814
runway                 535947.972371
                             average_CI
fclass                                 
burial_site                5.818399e+03
garden                     1.274380e+02
leisure_park               3.483240e+04
protected_provincial_park  1.362123e+08
          average_CI
fclass              
parking  1842.568347
Skipping /content/drive/My Drive/Shapefiles/NL_roads.gpkg (fields not found)
                           average_CI
fclass                               
farmland                 75637.069847
farmy